# Azure AI Agents Playground: Setup from Scratch (0 to 1) - Using Azure SDK

Welcome to your Azure AI Agents Playground! This notebook will guide you through setting up your first Azure AI Agent from complete scratch using the Azure Management Libraries for Python (Azure SDK).

**What we'll cover:**

1.  **Prerequisites:**  Tools and libraries you'll need.
2.  **Resource Group Setup:** Organizing your Azure resources using Azure SDK.
3.  **Bicep Template: The Resource Foundry:**  Defining our Azure AI Agent Service infrastructure with Bicep.
4.  **Deploying the Bicep Template:**  Creating the resources in Azure using Azure SDK.
5.  **Next Steps:** Where to go from here and how to explore your agent playground.

**Goal:** By the end of this notebook, you'll have a foundational Azure AI Agent Service instance ready for you to build upon in your `azure-ai-agents-playground` repository, all managed through Python code using the Azure SDK.

## 1. Prerequisites

Before we begin, make sure you have the following:

*   **Azure Subscription:** You'll need an active Azure subscription. If you don't have one, you can [create a free Azure account](https://azure.microsoft.com/free/).
*   **Python Environment:**  Python 3.8 or later is recommended.
*   **Jupyter Notebook:**  You are already here! If you need to install it, follow the instructions [here](https://jupyter.org/install).
*   **Azure CLI (Optional, for initial authentication):** While we'll be using the Azure SDK for resource creation, having Azure CLI installed can simplify initial authentication to Azure from your local environment. Instructions can be found [here](https://docs.microsoft.com/cli/azure/install-azure-cli).
*   **Bicep Extension for VS Code (Recommended):** Using VS Code with the Bicep extension will greatly enhance your experience when working with the Bicep template. Get it [here](https://marketplace.visualstudio.com/items?itemName=ms-azuretools.vscode-bicep).

**Install Azure Management Libraries:**

Run the following command to install the necessary Azure SDK libraries:


![Console CLI](../../data/img/01-1.png)

```python
!pip install azure-mgmt-resource azure-identity

In [ ]:
!pip install azure-mgmt-resource azure-identity

## 2. Authentication

To interact with Azure, we need to authenticate. We'll use `DefaultAzureCredential` from the `azure-identity` library, which intelligently handles authentication in various environments (e.g., your local machine using Azure CLI credentials, or Managed Identity within Azure).

**Action:**

Replace the placeholder `YOUR_AZURE_SUBSCRIPTION_ID` with your actual Azure Subscription ID. You can find your Subscription ID in the [Azure Portal](https://portal.azure.com) by searching for 'Subscriptions'.

In [2]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

# Authenticate using DefaultAzureCredential
credential = DefaultAzureCredential()

# Replace with your subscription ID
subscription_id = "ee787b9b-a25f-4d20-86e9-45fcea5253dd"  #  <----  IMPORTANT:  REPLACE THIS!

# Create Resource Management Client
resource_client = ResourceManagementClient(credential, subscription_id)

## 3. Resource Group Setup

Azure resources are organized into Resource Groups. We'll create a dedicated Resource Group for our Azure AI Agent Playground using the Azure SDK.

**Naming Convention:**

We'll use an intuitive naming convention for our Resource Group:
`rg-<environment>-<purpose>-<location_abbrev>`

For example, if you are creating a development environment in West US 2 for your AI Agents Playground, the Resource Group name would be:
`rg-dev-ai-agents-playground-westus2`

**Action:**

Replace the placeholders below with your desired values and run the cell to create your Resource Group using the Azure SDK.

In [5]:
resource_group_name = "rg-dev-ai-agents-playground-eastus2"  # Example name
resource_group_location = "eastus2"  # Example location (e.g., westus2, eastus, uksouth)

from azure.mgmt.resource.resources.models import ResourceGroup

# Define the Resource Group parameters
resource_group_params = ResourceGroup(location=resource_group_location)

try:
    # Create the Resource Group
    rg_result = resource_client.resource_groups.create_or_update(
        resource_group_name, resource_group_params
    )
    print(f"Resource Group '{resource_group_name}' created successfully in '{resource_group_location}'.")
    print(f"\nResource Group Details:\n{rg_result}")  # Print Resource Group details (optional)

except Exception as e:
    print(f"Error creating Resource Group '{resource_group_name}':")
    print(e)

Resource Group 'rg-dev-ai-agents-playground-eastus2' created successfully in 'eastus2'.

Resource Group Details:
{'additional_properties': {}, 'id': '/subscriptions/ee787b9b-a25f-4d20-86e9-45fcea5253dd/resourceGroups/rg-dev-ai-agents-playground-eastus2', 'name': 'rg-dev-ai-agents-playground-eastus2', 'type': 'Microsoft.Resources/resourceGroups', 'properties': <azure.mgmt.resource.resources.v2022_09_01.models._models_py3.ResourceGroupProperties object at 0x000001F41F93E850>, 'location': 'eastus2', 'managed_by': None, 'tags': None}


## 4. Bicep Template: The Resource Foundry

We'll use the same Bicep template (`main.bicep`) as before to define our Azure AI Agent Service infrastructure.  Bicep provides a declarative way to define your Azure resources.

**Our Bicep Template (`main.bicep`):**


## 5. Deploying the Bicep Template using Azure SDK

Now we'll deploy the Bicep template to create the Azure AI Agent Service in your Resource Group, this time using the Azure SDK for Python.

**Action:**

Run the cell below to deploy the `main.bicep` template using Azure SDK.  Make sure `main.bicep` is in the same directory as your Jupyter Notebook.

**Important:** Ensure the `resource_group_name` variable in the code below matches the name of the Resource Group you created in step 3.

In [ ]:
resource_group_name = "rg-dev-ai-agents-playground-eastus2" # Ensure this matches your Resource Group name!

import json
from azure.mgmt.resource.resources.models import Deployment, DeploymentProperties, TemplateLink

# 1. Read Bicep template content
with open("main.bicep", "r") as f:
    bicep_template_content = f.read() # Read Bicep as string content

# 2. Define deployment parameters
deployment_name = "bicepDeployment-" + resource_group_name # Unique deployment name
template_parameters = {
    "resourceGroupName": {"value": resource_group_name} # Pass Resource Group name as parameter
}

deployment_properties = DeploymentProperties(
    template_content=bicep_template_content, # Use loaded Bicep content as string
    parameters=template_parameters,
    mode="Incremental" # or "Complete" deployment mode as needed
)

deployment = Deployment(properties=deployment_properties)


try:
    # 3. Deploy the Bicep template using Resource Management Client
    deployment_result_poller = resource_client.deployments.begin_create_or_update(
        resource_group_name, deployment_name, deployment
    )
    deployment_result = deployment_result_poller.result()  # Wait for deployment to complete

    print("\nBicep template deployed successfully using Azure SDK!\n")
    # Print deployment details (optional)
    # print(f"Deployment Details:\n{deployment_result}")

    # Extract and print outputs from the deployment (if any)
    if deployment_result.properties and deployment_result.properties.outputs:
        outputs = deployment_result.properties.outputs
        print("\nDeployment Outputs:")
        for output_name, output_value in outputs.items():
            print(f"  {output_name}: {output_value['value']}")


except Exception as e:
    print(f"Error deploying Bicep template using Azure SDK:")
    print(e)

template_content is not a known attribute of class <class 'azure.mgmt.resource.resources.v2022_09_01.models._models_py3.DeploymentProperties'> and will be ignored


Error deploying Bicep template using Azure SDK:
(InvalidDeployment) The deployment 'bicepDeployment-rg-dev-ai-agents-playground-westus2' must have either the TemplateLink or Template property set. Please see https://aka.ms/arm-module for usage details.
Code: InvalidDeployment
Message: The deployment 'bicepDeployment-rg-dev-ai-agents-playground-westus2' must have either the TemplateLink or Template property set. Please see https://aka.ms/arm-module for usage details.
